In [39]:
import pandas as pd
import numpy as np

In [40]:
dataset_df = pd.read_csv("/content/dataset.csv")

In [41]:
dataset_df.head(10)

,Unnamed: 0,harmful,tweet
0,0,0,@user when a father is dysfunctional and is s...
1,1,0,@user @user thanks for #lyft credit i can't us...
2,2,0,bihday your majesty
3,3,0,#model i love u take with u all the time in ...
4,4,0,factsguide: society now #motivation
5,5,0,[2/2] huge fan fare and big talking before the...
6,6,0,@user camping tomorrow @user @user @user @use...
7,7,0,the next school year is the year for exams.ð...
8,8,0,we won!!! love the land!!! #allin #cavs #champ...
9,9,0,@user @user welcome here ! i'm it's so #gr...


In [42]:
dataset_df.drop(columns=["Unnamed: 0"])

,harmful,tweet
0,0,@user when a father is dysfunctional and is s...
1,0,@user @user thanks for #lyft credit i can't us...
2,0,bihday your majesty
3,0,#model i love u take with u all the time in ...
4,0,factsguide: society now #motivation
...,...,...
56740,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
56741,1,"you've gone and broke the wrong heart baby, an..."
56742,1,young buck wanna eat!!.. dat nigguh like I ain...
56743,1,youu got wild bitches tellin you lies


In [43]:
dataset_df.shape

(56745, 3)

In [44]:
dataset_df['harmful'].value_counts()

,count
harmful,
0,32592
1,24153


In [45]:
for _ in range(10):
  random_index = np.random.randint(0, len(dataset_df))
  random_data = dataset_df.iloc[random_index]
  print(random_data["tweet"], random_data["harmful"])

cravings satisfied. #cheeseburger  #mcdonalds #fries #mcdo   ðð  0
remember the day when people on #bbuk went on to become reality tv stars, no it seems reality tv stars become big brother contestants.   0
bitch niggas owe me money 1
do you love suspenders? we do!  pc:  katy for #heasfoto #love #instagood #cute #photooftheday   #beautiful...  0
@YourDudeFerg like a tree in this bitch? Asshat. 1
&#8220;@boojieque: &#128514;&#128169;RT @WebbMadeThis: Lemme find out you niggaz letting bitches shit on yo faces&#8221; &#128557; niggas is weirdos 1
RT @XANAXDREAMS: if u know how to French inhale ur a bad bitch 1
  #spring #equinox! mt calm end to #winter #after #months of &amp;  0
RT @SweetiePaii: RT if you listen to these bands &#128522;

A7X
PTV
SWS
ATL
BVB
ADTR
BMTH
YMAS
FOB
AA
Blink 182
TDG
SS
FIR
MCR

Follow me &amp; everyon&#8230; 0
RT @Shalewis09: Larry Elder is an Uncle Tom. I cannot believe how he is degrading a child. 1


Preprocessing

In [46]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [47]:
x = dataset_df["tweet"].values
y = dataset_df["harmful"].values

# The .values attribute converts the pandas Series (each column in a DataFrame) into a NumPy array. This is useful when you need to perform operations that require NumPy arrays or when you need to pass the data into machine learning models or other libraries that expect NumPy arrays as input.

In [48]:
print(x)

[' @user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run'
 "@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked"
 '  bihday your majesty' ...
 'young buck wanna eat!!.. dat nigguh like I aint fuckin dis up again'
 'youu got wild bitches tellin you lies'
 '~~Ruffled | Ntac Eileen Dahlia - Beautiful color combination of pink, orange, yellow &amp; white. A Coll http://t.co/H0dYEBvnZB']


In [49]:
sequences = [sequence for sequence in x]
model_inputs = tokenizer(sequences, padding=True, return_tensors="tf")

In [50]:
type(sequences)

list

In [51]:
type(x)

numpy.ndarray

In [52]:
type(model_inputs)

transformers.tokenization_utils_base.BatchEncoding

In [53]:
import tensorflow as tf

In [54]:
dataset = tf.data.Dataset.from_tensor_slices((model_inputs['input_ids'], y))

In [55]:
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

In [56]:
# Training pipeline

train = dataset.take(int(len(dataset)*0.7))
val = dataset.skip(int(len(dataset)*0.7)).take(int(len(dataset)*0.2))
test = dataset.skip(int(len(dataset)*0.9)).take(int(len(dataset)*0.1))

In [57]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Bidirectional

model = Sequential(name="tweet-classifier")
model.add(Embedding(len(tokenizer.get_vocab()), 32))
model.add(Bidirectional(LSTM(32, activation='tanh')))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

In [58]:

len(tokenizer.get_vocab())

30522

In [59]:
len(tokenizer)

30522